# ResNet18

Imports

In [ ]:
import torch
# import torchvision

# import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# from torch.utils.tensorboard import SummaryWriter

## Implementation

Raw implementation taken from PyTorch Git Hub

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        
        super(Block, self).__init__()
        self.num_layers = 18

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet18(nn.Module):
    def __init__(self, block, image_channels, num_classes):

        super(ResNet18, self).__init__()
 
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNetLayers
        self.layer1 = self.make_layers(block, 2, intermediate_channels=64, stride=1)
        self.layer2 = self.make_layers(block, 2, intermediate_channels=128, stride=2)
        self.layer3 = self.make_layers(block, 2, intermediate_channels=256, stride=2)
        self.layer4 = self.make_layers(block, 2, intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def make_layers(self, block, num_residual_blocks, intermediate_channels, stride):
        layers = []

        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, intermediate_channels, kernel_size=1, stride=stride),
                                            nn.BatchNorm2d(intermediate_channels))
        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))
        self.in_channels = intermediate_channels  # 256
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels)) # 256 -> 64, 64*4 (256) again
        return nn.Sequential(*layers)

## Test

In [ ]:
net = ResNet18(Block, 3, 1000)
y = net(torch.randn(4, 3, 224, 224)).to(DEVICE)
print(y.size())
print(y)

torch.Size([4, 1000])
tensor([[-0.1420, -0.2695,  0.5869,  ..., -0.3288,  0.3506,  0.4209],
        [-0.2983, -0.2310,  0.6669,  ..., -0.3002,  0.6345,  0.5241],
        [-0.2494, -0.1921,  0.5339,  ..., -0.2300,  0.4833,  0.4644],
        [-0.2014, -0.2371,  0.6730,  ..., -0.2191,  0.4419,  0.4853]],
       device='cuda:0', grad_fn=<CopyBackwards>)
